### 1. Imports
Import all the libraries and modules needed

In [1]:
import pandas as pd

### 2. Datasets
Load only the CSV files that have information we need for predictions, and leave out those that don’t add value:
1. **Load core tables**: Read in `studentVle.csv`, `assessments.csv`, `studentAssessment.csv`, `studentInfo.csv` into pandas DataFrames
2. **Skip unneeded files**: Exclude `vle.csv` and `studentRegistration.csv` as they lack meaningful features for our predicitions
3. **Skip redundant table**: Omit `courses.csv` because its contents are already represented by other files

In [2]:
# Load datasets
df_info             = pd.read_csv('./databases/OULAD/raw_data/studentInfo.csv')
df_assess           = pd.read_csv('./databases/OULAD/raw_data/assessments.csv')
df_student_assess   = pd.read_csv('./databases/OULAD/raw_data/studentAssessment.csv')
df_student_vle      = pd.read_csv('./databases/OULAD/raw_data/studentVle.csv')
#df_vle              = pd.read_csv('./databases/OULAD/raw_data/vle.csv')
#df_reg              = pd.read_csv('./databases/OULAD/raw_data/studentRegistration.csv')
#df_courses          = pd.read_csv('./databases/OULAD/raw_data/courses.csv')


### 3. Preprocessing Dataset

#### 3.1 IMD_band fix and merge assessments
Fix imd_band, prepare assessment data and create helper list by:  
1. **Fix `imd_band`**: Append a “%” to any `imd_band` entries that lack one (e.g., turning “10–20” into “10–20%”) 
2. **Merge assessment tables**: Join the `assessments` DataFrame with `studentAssessment` on `id_assessment` to bring together assessments, deadlines and student scores
3. **Helper list**: Create student_module_list with `id_student`, `code_module`, `code_presentation` for easier selection later

In [8]:
# Fix imd_band values
def fix_imd(s):
    if s.endswith('%'):
        return s
    return s + '%'

df_info['imd_band'] = df_info['imd_band'].astype(str).apply(fix_imd)


# Merge Asessments with Student Assessments
df_assessments = pd.merge(df_assess, df_student_assess, on='id_assessment')

student_module_list = ['id_student', 'code_module', 'code_presentation']

#### 3.2 Late Submissions
Here we quantify each student’s tendency to miss deadlines by:  
1. **Identify late records**: Filter assessment submissions where the submission date falls after the deadline
2. **Aggregate late count**: Sum the number of late submissions for each student–module pairing

In [9]:
# Late submissions
df_assessments['late_submission'] = (
    df_assessments['date_submitted'] > df_assessments['date']
)
late_counts = (
    df_assessments
      .groupby(student_module_list)['late_submission']
      .sum()
      .reset_index(name='late_submissions_in_module')
)

#### 3.3 Average Assessment Scores
Measure each student’s overall performance within a module by:  
1. **Assessment count per module**: Determine how many assessments are assigned in each module
2. **Total score per student**: Sum up every assessment score a student has achieved in that module
3. **Average score calculation**: Compute the student’s mean by dividing their total score by the module’s assessment count
4. **Column selection**: Keep only the key fields `id_student`, `code_module`, `code_presentation`, and `avg_score_in_module`

In [10]:
# Average scores
total_assessments = (
    df_assess
      .groupby(['code_module','code_presentation'])['id_assessment']
      .nunique()
      .reset_index(name='total_assessments_in_module')
)

sum_scores = (
    df_assessments
      .groupby(student_module_list)['score']
      .sum()
      .reset_index(name='sum_scores_in_module')
)

avg_scores = (
    sum_scores
      .merge(total_assessments, on=['code_module','code_presentation'], how='left')
)
avg_scores['sum_scores_in_module'] = avg_scores['sum_scores_in_module'].fillna(0)
avg_scores['avg_score_in_module'] = (
    avg_scores['sum_scores_in_module']
    / avg_scores['total_assessments_in_module']
)
avg_scores = avg_scores[
    ['id_student', 'code_module', 'code_presentation', 'avg_score_in_module']
]

#### 3.4 Student Engagement
Evaluate each student’s engagement:  
1. **Total clicks**: Sum all clicks for each student–module pairing
2. **Clicks per day**: Daily counts to capture how many clicks each student generates on a given day
3. **Average clicks per day**: Compute the mean of these daily counts for each student to serve as a normalized measure of their typical online activity
4. **Number of active days**: Sum of all the days where the student clicked atleast once. (active)

In [11]:
# Click counts
click_count = (
    df_student_vle
        .groupby(student_module_list)['sum_click']
        .sum()
        .reset_index(name='sum_clicks_in_module')
)

# Clicks per session
clicks_per_session = (
    df_student_vle
      .groupby(['id_student', 'code_module', 'code_presentation', 'date'])['sum_click']
      .sum()
      .reset_index(name='clicks_per_session')
)
display(clicks_per_session)
# Average clicks per session (Day)
avg_clicks_per_session = (
    clicks_per_session
      .groupby(student_module_list)['clicks_per_session']
      .mean()
      .reset_index(name='avg_clicks_per_session')
)

# Number of active days
num_active_days = (
    df_student_vle
      .groupby(student_module_list)['date']
      .nunique()
      .reset_index(name='num_active_days')
)

,id_student,code_module,code_presentation,date,clicks_per_session
0,6516,AAA,2014J,-23,28
1,6516,AAA,2014J,-22,82
2,6516,AAA,2014J,-20,41
3,6516,AAA,2014J,-17,7
4,6516,AAA,2014J,-12,2
...,...,...,...,...,...
1808114,2698588,BBB,2014J,240,3
1808115,2698588,BBB,2014J,244,3
1808116,2698588,BBB,2014J,248,5
1808117,2698588,BBB,2014J,250,2


### 4. Merge
In this step, we bring together all the feature tables into our master dataset (`big_df`) by:  
1. **Joining feature tables**: Left-merge each table `avg_scores`, `late_counts`, `click_count`, `num_active_days` and `avg_clicks_per_session` onto `big_df` using the keys `(id_student, code_module, code_presentation)`
2. **Preserving complete records**: Use left joins to keep every student entry in `big_df`, even if a particular feature table has no matching rows

In [12]:
# Merge into big DataFrame
big_df = df_info
big_df = (
    big_df
    .merge(late_counts, on=student_module_list, how='left')
    .merge(avg_scores, on=student_module_list, how='left')
    .merge(click_count, on=student_module_list, how='left')
    .merge(avg_clicks_per_session, on=student_module_list, how='left')
    .merge(num_active_days, on=student_module_list, how='left')
)
display(big_df)

,code_module,code_presentation,id_student,gender,region,highest_education,imd_band,age_band,num_of_prev_attempts,studied_credits,disability,final_result,late_submissions_in_module,avg_score_in_module,sum_clicks_in_module,avg_clicks_per_session,num_active_days
0,AAA,2013J,11391,M,East Anglian Region,HE Qualification,90-100%,55<=,0,240,N,Pass,0.0,68.333333,934.0,23.350000,40.0
1,AAA,2013J,28400,F,Scotland,HE Qualification,20-30%,35-55,0,60,N,Pass,2.0,55.333333,1435.0,17.937500,80.0
2,AAA,2013J,30268,F,North Western Region,A Level or Equivalent,30-40%,35-55,0,60,Y,Withdrawn,NaN,NaN,281.0,23.416667,12.0
3,AAA,2013J,31604,F,South East Region,A Level or Equivalent,50-60%,35-55,0,60,N,Pass,0.0,63.333333,2158.0,17.544715,123.0
4,AAA,2013J,32885,F,West Midlands Region,Lower Than A Level,50-60%,0-35,0,60,N,Pass,5.0,45.333333,1034.0,14.771429,70.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32588,GGG,2014J,2640965,F,Wales,Lower Than A Level,10-20%,0-35,0,30,N,Fail,NaN,NaN,41.0,8.200000,5.0
32589,GGG,2014J,2645731,F,East Anglian Region,Lower Than A Level,40-50%,35-55,0,30,N,Distinction,2.0,79.300000,893.0,19.844444,45.0
32590,GGG,2014J,2648187,F,South Region,A Level or Equivalent,20-30%,0-35,0,30,Y,Pass,0.0,69.000000,312.0,13.000000,24.0
32591,GGG,2014J,2679821,F,South East Region,Lower Than A Level,90-100%,35-55,0,30,N,Withdrawn,0.0,18.300000,275.0,21.153846,13.0


### 5. Clean up data
We finalize our dataset by:  
1. **Removing incomplete records**: Drop any rows that have missing values to ensure every record is fully populated
2. **Round numerics**: Round all numeric features to two decimal places for consistency and readability

In [13]:
# Clean up the data from rows with missing values
big_df = big_df.dropna()

# Round the values to 2 decimal places
big_df = big_df.round(2)

#### 6. Save DataFrame as CSV
In this final step, we create separate csv files for each code-module combination:
1. **Partitioning the data**: Group `big_df` by `code_module` and `code_presentation`
2. **File naming**: Construct filenames in the format `{module}_{presentation}.csv`
3. **Saving**: Write each subgroup to its own CSV under `./databases/OULAD/`

In [ ]:
# Split and save CSVs by code_module and code_presentation
for (module, presentation), group in big_df.groupby(['code_module', 'code_presentation']):
    filename = f"{module}_{presentation}.csv"
    group.to_csv(f'./databases/OULAD/preprocessed_data/{filename}', index=False)

: 